In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import numpy as np
import pandas as pd

from collections import Counter

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier

In [2]:
math_file_path = os.path.join("Student_alcohol_consumption", "student-mat.csv")
math = pd.read_csv(math_file_path)

por_file_path = os.path.join("Student_alcohol_consumption", "student-por.csv")
por = pd.read_csv(por_file_path)

In [3]:
# clean bucket the grades
# 10 and above is a pass
# 9 and below is a fail

def encode_grades(data):
    # bucket the grades into passing(1) and failling(0)
    
    # math
    # failling
    data.loc[(data["G1"] < 10), "G1"] = 0
    data.loc[(data["G2"] < 10), "G2"] = 0
    data.loc[(data["G3"] < 10), "G3"] = 0

    #passing
    data.loc[(data["G1"] >= 10), "G1"] = 1
    data.loc[(data["G2"] >= 10), "G2"] = 1
    data.loc[(data["G3"] >= 10), "G3"] = 1
    
    display(data)
    
    return

encode_grades(math)
encode_grades(por)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,0,0,0
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,0,0,0
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,0,0,1
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,1,1,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,20,U,LE3,A,2,2,services,services,...,5,5,4,4,5,4,11,0,0,0
391,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,3,1,1,1
392,MS,M,21,R,GT3,T,1,1,other,other,...,5,5,3,3,3,3,3,1,0,0
393,MS,M,18,R,LE3,T,3,2,services,other,...,4,4,1,3,4,5,0,1,1,1


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,1,1
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,0,1,1
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,1,1,1
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,1,1,1
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,1,1,1
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,1,1,1
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,1,1,0
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,1,1,1


In [4]:
def encode_features(data):

    for i in data.columns.tolist():
        le = LabelEncoder()
        data[i] = le.fit_transform(data[i])
        
    display(data)
    
    return

encode_features(math)
encode_features(por)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0,0,3,1,0,0,4,4,0,4,...,3,2,3,0,0,2,6,0,0,0
1,0,0,2,1,0,1,1,1,0,2,...,4,2,2,0,0,2,4,0,0,0
2,0,0,0,1,1,1,1,1,0,2,...,3,2,1,1,2,2,10,0,0,1
3,0,0,0,1,0,1,4,2,1,3,...,2,1,1,0,0,4,2,1,1,1
4,0,0,1,1,0,1,3,3,2,2,...,3,2,1,0,1,4,4,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,1,1,5,1,1,0,2,2,3,3,...,4,4,3,3,4,3,11,0,0,0
391,1,1,2,1,1,1,3,1,3,3,...,1,3,4,2,3,1,3,1,1,1
392,1,1,6,0,0,1,1,1,2,2,...,4,4,2,2,2,2,3,1,0,0
393,1,1,3,0,1,1,3,2,3,2,...,3,3,0,2,3,4,0,1,1,1


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,0,0,3,1,0,0,4,4,0,4,...,3,2,3,0,0,2,4,0,1,1
1,0,0,2,1,0,1,1,1,0,2,...,4,2,2,0,0,2,2,0,1,1
2,0,0,0,1,1,1,1,1,0,2,...,3,2,1,1,2,2,6,1,1,1
3,0,0,0,1,0,1,4,2,1,3,...,2,1,1,0,0,4,0,1,1,1
4,0,0,1,1,0,1,3,3,2,2,...,3,2,1,0,1,4,0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,1,0,4,0,0,1,2,3,3,2,...,4,3,1,0,1,4,4,1,1,1
645,1,0,3,1,1,1,3,1,4,3,...,3,2,3,0,0,0,4,1,1,1
646,1,0,3,1,0,1,1,1,2,2,...,0,0,0,0,0,4,6,1,1,0
647,1,1,2,1,1,1,3,1,3,3,...,1,3,4,2,3,1,6,1,1,1


In [5]:
def over_sample(df, drop, target):
    
    # split the table into features and outcomes
    x_cols = [i for i in math.columns if i not in drop]
    X = df[x_cols]
    y = df[target]

    # split features and outcomes into train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    # oversample to make up for the low number of risky loans
    ros = RandomOverSampler(random_state=1)
    X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

    model = LogisticRegression(solver='lbfgs', random_state=1)
    model.fit(X_resampled, y_resampled)
    y_predictions = model.predict(X_test)

    # Calculating the accuracy score.
    acc_score = balanced_accuracy_score(y_test, y_predictions)
    
    return acc_score*100

In [6]:
def under_sample(df, drop, target):
    
    # split the table into features and outcomes
    x_cols = [i for i in math.columns if i not in drop]
    X = df[x_cols]
    y = df[target]

    # split features and outcomes into train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    ros = RandomUnderSampler(random_state=1)
    X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

    model = LogisticRegression(solver='lbfgs', random_state=1)
    model.fit(X_resampled, y_resampled)
    y_predictions = model.predict(X_test)
    
    # Calculating the accuracy score.
    acc_score = balanced_accuracy_score(y_test, y_predictions)
    
    return acc_score*100

In [7]:
def cluster(df, drop, target):
    
    # split the table into features and outcomes
    x_cols = [i for i in math.columns if i not in drop]
    X = df[x_cols]
    y = df[target]
    
    # split features and outcomes into train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    cc = ClusterCentroids(random_state=1)
    X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

    model = LogisticRegression(solver='lbfgs', random_state=1)
    model.fit(X_resampled, y_resampled)

    y_predictions = model.predict(X_test)

    # Calculating the accuracy score.
    acc_score = balanced_accuracy_score(y_test, y_predictions)
    
    return acc_score*100

In [8]:
def smoteen(df, drop, target):
    
    # split the table into features and outcomes
    x_cols = [i for i in math.columns if i not in drop]
    X = df[x_cols]
    y = df[target]

    # split features and outcomes into train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    smote_enn = SMOTEENN(random_state=0)
    X_resampled, y_resampled = smote_enn.fit_resample(X, y)

    model = LogisticRegression(solver='lbfgs', random_state=1)
    model.fit(X_resampled, y_resampled)
    y_predictions = model.predict(X_test)

    # Calculating the accuracy score.
    acc_score = balanced_accuracy_score(y_test, y_predictions)
    
    return acc_score*100

In [9]:
def random_forest(df, drop, target):

    # split the table into features and outcomes
    x_cols = [i for i in math.columns if i not in drop]
    X = df[x_cols]
    y = df[target]
    
    # split features and outcomes into train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    brf = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
    brf.fit(X_train, y_train)
    y_predictions = brf.predict(X_test)

    feature_importance = sorted(zip(brf.feature_importances_, X.columns.tolist()))[::-1]

    # Calculating the accuracy score.
    acc_score = balanced_accuracy_score(y_test, y_predictions)

    # Displaying results
    print("Feature Importance")
    for i in feature_importance:
        print(i)
    
    return acc_score*100

In [10]:
def easy_ensemble_classifier(df, drop, target):
    
    # split the table into features and outcomes
    x_cols = [i for i in math.columns if i not in drop]
    X = df[x_cols]
    y = df[target]
    
    # split features and outcomes into train and test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    eec = EasyEnsembleClassifier(n_estimators=100, random_state=0)
    eec.fit(X_train, y_train)
    y_predictions = eec.predict(X_test)

    # Calculating the accuracy score.
    acc_score = balanced_accuracy_score(y_test, y_predictions)
    
    return acc_score*100

# We do not want to have to run 6 functions for each target so lets make a function that will handle it.

In [11]:
def model_summary(df, drop, target):
    
    # make a dataframe to neatly organize our results
    machine_learning_summary = pd.DataFrame(
        {
            "Over Sampling": [over_sample(df, drop, target)],
            "Under Sampling": [under_sample(df, drop, target)],
            "Cluster Centroids": [cluster(df, drop, target)],
            "SMOTEENN": [smoteen(df, drop, target)],
            "Random Forest": [random_forest(df, drop, target)],
            "Easy Ensemble Classifier": [easy_ensemble_classifier(df, drop, target)]
        }
    )
    
    # format the accuracy scores to make them easier to read and more descriptive
    machine_learning_summary["Over Sampling"] = machine_learning_summary["Over Sampling"].map("{:.1f}%".format)
    machine_learning_summary["Under Sampling"] = machine_learning_summary["Under Sampling"].map("{:.1f}%".format)
    machine_learning_summary["Cluster Centroids"] = machine_learning_summary["Cluster Centroids"].map("{:.1f}%".format)
    machine_learning_summary["SMOTEENN"] = machine_learning_summary["SMOTEENN"].map("{:.1f}%".format)
    machine_learning_summary["Random Forest"] = machine_learning_summary["Random Forest"].map("{:.1f}%".format)
    machine_learning_summary["Easy Ensemble Classifier"] = machine_learning_summary["Easy Ensemble Classifier"].map("{:.1f}%".format)
    
    # change the index name it more clearly state that it is the accuracy scores being displayed
    machine_learning_summary = machine_learning_summary.rename(index={0: "Accuracy Score"})
    # show us the dataframe
    display(machine_learning_summary)
    
    return 

In [12]:
# different columns to drop depending on which target we are using
# we are not dropping previous grades because it is a reasonable expectation to have those data points sequential trimesters
G1 = ("G1", "G2", "G3")
G2 = ("G2", "G3")
G3 = ("G3")

#model_summary(df=math, drop=G1, target="G1")
#model_summary(df=math, drop=G2, target="G2")
#model_summary(df=math, drop=G3, target="G3")

#model_summary(df=por, drop=G1, target="G1")
#model_summary(df=por, drop=G2, target="G2")
#model_summary(df=por, drop=G3, target="G3")

# Slightly unnecessary, but it would be even more nice to only have to run one function to check all the targets.

In [13]:
def build_models():

    # different columns to drop depending on which target we are using
    # we are not dropping previous grades because it is a reasonable expectation to have those data points sequential trimesters
    G1 = ("G1", "G2", "G3")
    G2 = ("G2", "G3")
    G3 = ("G3")

    # formatting the way that the results are displayed
    # line breaks and labels are added to improve readablity
    
    # math models
    # G1
    print("-"*125 + "\n" + "Math G1 Summary Table\n")
    model_summary(df=math, drop=G1, target="G1")
    # G2
    print("-"*125 + "\n" "Math G2 Summary Table\n")
    model_summary(df=math, drop=G2, target="G2")
    # G3
    print("-"*125 + "\n" "Math G3 Summary Table\n")
    model_summary(df=math, drop=G3, target="G3")

    # por models
    # G1
    print("-"*125 + "\n" "Portuguese G1 Summary Table\n")
    model_summary(df=por, drop=G1, target="G1")
    # G2
    print("-"*125 + "\n" "Portuguese G2 Summary Table\n")
    model_summary(df=por, drop=G2, target="G2")
    # G3
    print("-"*125 + "\n" "Portuguese G3 Summary Table\n")
    model_summary(df=por, drop=G3, target="G3")
    
    return

In [14]:
build_models()

-----------------------------------------------------------------------------------------------------------------------------
Math G1 Summary Table

Feature Importance
(0.07552015588930218, 'absences')
(0.0741165088766574, 'Fedu')
(0.060873078960852305, 'failures')
(0.053474408784157504, 'studytime')
(0.05236555674811134, 'Mjob')
(0.04818087755809755, 'freetime')
(0.0474152549255183, 'Medu')
(0.0458958103540737, 'age')
(0.043323686736156955, 'health')
(0.04246381335711076, 'Walc')
(0.040331573845309214, 'goout')
(0.03700496526341975, 'reason')
(0.03539569852804747, 'Dalc')
(0.03447889596108349, 'famrel')
(0.030414449343871008, 'Fjob')
(0.026008284858731967, 'guardian')
(0.025681861660632268, 'traveltime')
(0.02405448454967536, 'schoolsup')
(0.021293562166722343, 'sex')
(0.020586250395254812, 'paid')
(0.020534523118049552, 'famsup')
(0.019635232328878875, 'famsize')
(0.01877966589770088, 'nursery')
(0.017732042723502848, 'activities')
(0.0176433631554651, 'address')
(0.01630976107463790

,Over Sampling,Under Sampling,Cluster Centroids,SMOTEENN,Random Forest,Easy Ensemble Classifier
Accuracy Score,56.6%,64.0%,55.8%,65.4%,60.2%,71.0%


-----------------------------------------------------------------------------------------------------------------------------
Math G2 Summary Table

Feature Importance
(0.28863992658328863, 'G1')
(0.052207841118592294, 'absences')
(0.04571837557279182, 'goout')
(0.03687566644555507, 'age')
(0.03585529646619073, 'Medu')
(0.0354552788167293, 'failures')
(0.0354028078691229, 'Mjob')
(0.03479662462056467, 'Walc')
(0.03335591122582404, 'studytime')
(0.030952817785297432, 'health')
(0.03000288862847583, 'reason')
(0.029867794158010785, 'Fedu')
(0.02905847696340761, 'freetime')
(0.02730717348877115, 'famrel')
(0.023468272345419282, 'Fjob')
(0.021571071085908926, 'Dalc')
(0.021480385029389978, 'traveltime')
(0.018800888605443215, 'romantic')
(0.016952610606687367, 'sex')
(0.015546496957724571, 'guardian')
(0.014981040096361788, 'internet')
(0.014727370340730914, 'nursery')
(0.01434080113794993, 'schoolsup')
(0.014184205540133242, 'address')
(0.014107237335899225, 'activities')
(0.0136420297663

,Over Sampling,Under Sampling,Cluster Centroids,SMOTEENN,Random Forest,Easy Ensemble Classifier
Accuracy Score,86.4%,88.7%,86.4%,88.6%,88.7%,87.1%


-----------------------------------------------------------------------------------------------------------------------------
Math G3 Summary Table

Feature Importance
(0.35239341535180474, 'G2')
(0.16140828041995653, 'G1')
(0.03765691304697964, 'age')
(0.03404896844677103, 'absences')
(0.03187628672835719, 'Mjob')
(0.026945336389663385, 'failures')
(0.025213612035328067, 'Fedu')
(0.023633294789985312, 'goout')
(0.022544619047961414, 'reason')
(0.02186733813776867, 'health')
(0.020782124659347482, 'Walc')
(0.019600043042871006, 'Fjob')
(0.018946511533406783, 'famrel')
(0.018467095470596467, 'studytime')
(0.018319505647276347, 'Medu')
(0.016396610112998138, 'freetime')
(0.015821512400383084, 'Dalc')
(0.015128232079602195, 'traveltime')
(0.012962806709991363, 'romantic')
(0.011830917202268976, 'paid')
(0.011426023837068217, 'schoolsup')
(0.011257580593511196, 'famsup')
(0.010033693155955401, 'guardian')
(0.008733541073861747, 'activities')
(0.008003051765965464, 'sex')
(0.007717559387730

...

,Over Sampling,Under Sampling,Cluster Centroids,SMOTEENN,Random Forest,Easy Ensemble Classifier
Accuracy Score,70.5%,72.5%,71.6%,71.5%,75.9%,76.8%


-----------------------------------------------------------------------------------------------------------------------------
Portuguese G2 Summary Table

Feature Importance
(0.22511796939297032, 'G1')
(0.06888819848964638, 'failures')
(0.04843889980403394, 'absences')
(0.03718721401588087, 'goout')
(0.036513869146892905, 'Fedu')
(0.03613692484559296, 'reason')
(0.035077171781173234, 'age')
(0.03507285505209215, 'studytime')
(0.03452395396895953, 'freetime')
(0.03412427608186755, 'Walc')
(0.03407126044412472, 'famrel')
(0.030756070317026526, 'health')
(0.030105204317525166, 'Medu')
(0.027688237671757192, 'Mjob')
(0.027100733431234092, 'school')
(0.02605367964595538, 'sex')
(0.025132421208691483, 'Dalc')
(0.02381162661784975, 'higher')
(0.023207296817018476, 'Fjob')
(0.021842109776796726, 'traveltime')
(0.01768730305646594, 'guardian')
(0.016043228319594566, 'activities')
(0.015939422187820428, 'nursery')
(0.015176837205396566, 'romantic')
(0.013983200345026996, 'address')
(0.0135979316

,Over Sampling,Under Sampling,Cluster Centroids,SMOTEENN,Random Forest,Easy Ensemble Classifier
Accuracy Score,77.9%,83.6%,68.2%,83.2%,81.7%,77.9%


-----------------------------------------------------------------------------------------------------------------------------
Portuguese G3 Summary Table

Feature Importance
(0.27988259873353316, 'G2')
(0.19674616316878907, 'G1')
(0.05310582757951958, 'failures')
(0.033765992230533455, 'school')
(0.02782349017464837, 'Walc')
(0.026999852301643697, 'absences')
(0.023534387026096903, 'goout')
(0.023396782529408102, 'age')
(0.022741923874442013, 'famrel')
(0.022375539795965172, 'Mjob')
(0.022230988799677047, 'Dalc')
(0.02139264796640146, 'reason')
(0.021072690382148953, 'freetime')
(0.019879591423207013, 'Fedu')
(0.01983249193726014, 'higher')
(0.019640503187246736, 'studytime')
(0.019442496530911405, 'Fjob')
(0.016919318909505047, 'health')
(0.016515597751944486, 'traveltime')
(0.014350166163519269, 'address')
(0.013701828818365656, 'Medu')
(0.011442576381402914, 'internet')
(0.01081105390503207, 'nursery')
(0.010512362140425306, 'sex')
(0.009613225748948342, 'romantic')
(0.0092336623597

,Over Sampling,Under Sampling,Cluster Centroids,SMOTEENN,Random Forest,Easy Ensemble Classifier
Accuracy Score,83.4%,89.7%,69.1%,89.7%,89.7%,88.2%


# Report
###### So I do not think it is a surprise to see that when we include previous grades that those are the most impactful on the accuracy of the model.
###### Likewise their number of absences and other failed classed are a good predictors across all targets.
###### Weekend alcohol consumption with going out also have a high correlation. This could be because socializing is cutting too much into study time or maybe there is a correlation between party goers and people that do not take school as serious.